In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import seaborn as sns
import pandas as pd
import numpy as np
import os, random, shutil, cv2
from shutil import copyfile, copy
from keras import applications,activations
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras import optimizers,utils
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,BatchNormalization,ZeroPadding2D, Input, SeparableConv2D
from keras.layers import Conv2D, Activation,MaxPooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Copy of df_with_ngram.csv')
df.head()

,Block,Author Name,Words per sentence,Commas per sentence,Excalimation per sentence,Questions per sentence,Salutation per sentence,He per sentence,She per sentence,ACD_bigram_score,CD_bigram_score,HGW_bigram_score,JA_bigram_score,JS_bigram_score,ACD_trigram_score,CD_trigram_score,HGW_trigram_score,JA_trigram_score,JS_trigram_score,ACD_bigram_count,CD_bigram_count,HGW_bigram_count,JA_bigram_count,JS_bigram_count
0,"[' produced by david brannan.', 'html versi...",Arthur Conan Doyle,14.5785,1.1255,0.0600,0.1175,0.1205,0.1515,1.8845,12556,16106,13578,1660,1326,0,0,0,0,0,40,20,37,7,13
1,"['if your peace should help another to peace, ...",Arthur Conan Doyle,12.6110,0.9550,0.1050,0.0820,0.0425,0.0705,1.5245,12597,28345,13128,1544,1395,0,0,0,0,0,35,31,35,7,10
2,['and so we must cut down every expense until ...,Arthur Conan Doyle,10.4260,0.8675,0.1015,0.1230,0.1050,0.0795,1.1125,12062,76079,20382,2534,1661,0,0,0,0,0,32,63,55,11,14
3,"['how mean of them!', 'dont let them have it, ...",Arthur Conan Doyle,15.8695,1.0500,0.0750,0.0685,0.0405,0.1200,2.2515,40894,22048,11816,1323,1450,887,0,0,0,0,103,22,34,6,13
4,['it will be remembered that the 46th brigade ...,Arthur Conan Doyle,28.9245,1.8555,0.0035,0.0005,0.0765,0.0905,4.8365,191159,1829,3882,460,214,4073,0,0,0,0,453,3,10,2,2


In [ ]:
selection = pd.read_csv('/content/drive/MyDrive/Final Sahred/X_train1.csv')
selection = selection.loc[:,'Words per sentence':]
X_train=selection.to_numpy()

result = np.where(X_train > 0.0000000001,X_train, -10)
# print(result)
np.log10(result, out=result, where=result > 0)
X_train[:,7:-1]=result[:,7:-1]
print(X_train)

[[1.66920000e+01 1.33250000e+00 8.00000000e-02 ... 2.06445799e+00
  1.50514998e+00 9.00000000e+00]
 [2.24855000e+01 1.98700000e+00 2.50000000e-02 ... 1.57978360e+00
  1.11394335e+00 1.60000000e+02]
 [1.40175000e+01 1.03250000e+00 6.20000000e-02 ... 1.66275783e+00
  7.78151250e-01 4.00000000e+00]
 ...
 [1.74202101e+01 1.89944972e+00 1.80090045e-01 ... 2.01283722e+00
  8.45098040e-01 6.00000000e+00]
 [1.75565000e+01 1.87350000e+00 1.28500000e-01 ... 1.99122608e+00
  1.47712125e+00 1.80000000e+01]
 [3.25810000e+01 2.92150000e+00 6.60000000e-02 ... 1.97312785e+00
  1.27875360e+00 4.80000000e+01]]


In [ ]:
selection=pd.read_csv('/content/drive/MyDrive/Final Sahred/X_test1.csv')
selection = selection.loc[:,'Words per sentence':]
X_test=selection.to_numpy()

result = np.where(X_test > 0.0000000001,X_test, -10)
# print(result)
np.log10(result, out=result, where=result > 0)
X_test[:,7:-1]=result[:,7:-1]
print(X_test)

[[2.05050000e+01 1.45900000e+00 4.90000000e-02 ... 1.86332286e+00
  4.77121255e-01 1.30000000e+01]
 [9.44450000e+00 9.65000000e-01 1.50500000e-01 ... 1.81291336e+00
  1.04139269e+00 3.00000000e+00]
 [2.01015000e+01 1.63400000e+00 1.12000000e-01 ... 1.34242268e+00
  1.65321251e+00 1.30000000e+01]
 ...
 [1.76560000e+01 1.67150000e+00 1.19000000e-01 ... 1.61278386e+00
  1.04139269e+00 2.60000000e+01]
 [9.23200000e+00 6.65500000e-01 1.04500000e-01 ... 1.83884909e+00
  9.03089987e-01 4.00000000e+00]
 [3.00390000e+01 1.49700000e+00 3.00000000e-03 ... 2.04532298e+00
  4.77121255e-01 4.50000000e+01]]


In [ ]:
selection = pd.read_csv('/content/drive/MyDrive/Final Sahred/y_train.csv')
y_train=selection.to_numpy()
print(y_train)

[['H. G. Wells']
 ['Jonathan Swift']
 ['Arthur Conan Doyle']
 ['Charles Dickens']
 ['Arthur Conan Doyle']
 ['Charles Dickens']
 ['Charles Dickens']
 ['Arthur Conan Doyle']
 ['H. G. Wells']
 ['Charles Dickens']
 ['Arthur Conan Doyle']
 ['Charles Dickens']
 ['Arthur Conan Doyle']
 ['H. G. Wells']
 ['Charles Dickens']
 ['Arthur Conan Doyle']
 ['Charles Dickens']
 ['Charles Dickens']
 ['Jonathan Swift']
 ['Arthur Conan Doyle']
 ['H. G. Wells']
 ['Charles Dickens']
 ['Charles Dickens']
 ['Jane Austen']
 ['Charles Dickens']
 ['Jane Austen']
 ['Jane Austen']
 ['Arthur Conan Doyle']
 ['Arthur Conan Doyle']
 ['Charles Dickens']
 ['H. G. Wells']
 ['Charles Dickens']
 ['H. G. Wells']
 ['Arthur Conan Doyle']
 ['Arthur Conan Doyle']
 ['Arthur Conan Doyle']
 ['Arthur Conan Doyle']
 ['H. G. Wells']
 ['Charles Dickens']
 ['H. G. Wells']
 ['H. G. Wells']
 ['H. G. Wells']
 ['Charles Dickens']
 ['Arthur Conan Doyle']
 ['Charles Dickens']
 ['H. G. Wells']
 ['H. G. Wells']
 ['Charles Dickens']
 ['H. G. Wel

In [ ]:
selection = pd.read_csv('/content/drive/MyDrive/Final Sahred/y_test.csv')
y_test=selection.to_numpy()
print(y_test)

[['Arthur Conan Doyle']
 ['Charles Dickens']
 ['Jane Austen']
 ['Arthur Conan Doyle']
 ['Arthur Conan Doyle']
 ['Charles Dickens']
 ['Charles Dickens']
 ['Arthur Conan Doyle']
 ['Charles Dickens']
 ['H. G. Wells']
 ['H. G. Wells']
 ['Arthur Conan Doyle']
 ['Jane Austen']
 ['Arthur Conan Doyle']
 ['Charles Dickens']
 ['Charles Dickens']
 ['Charles Dickens']
 ['Charles Dickens']
 ['Arthur Conan Doyle']
 ['Jane Austen']
 ['Charles Dickens']
 ['H. G. Wells']
 ['Charles Dickens']
 ['Arthur Conan Doyle']
 ['Arthur Conan Doyle']
 ['Charles Dickens']
 ['H. G. Wells']
 ['Charles Dickens']
 ['Charles Dickens']
 ['H. G. Wells']
 ['H. G. Wells']
 ['H. G. Wells']
 ['Arthur Conan Doyle']
 ['Arthur Conan Doyle']
 ['Charles Dickens']
 ['Charles Dickens']
 ['Charles Dickens']
 ['Charles Dickens']
 ['H. G. Wells']
 ['Charles Dickens']
 ['H. G. Wells']
 ['H. G. Wells']
 ['Arthur Conan Doyle']
 ['Charles Dickens']
 ['H. G. Wells']
 ['Charles Dickens']
 ['Charles Dickens']
 ['Charles Dickens']
 ['Arthur Co

In [ ]:
import sklearn
X_train=sklearn.preprocessing.normalize(X_train,axis=1) 
X_test=sklearn.preprocessing.normalize(X_test,axis=1)
print(X_train)

[[5.62288962e-01 4.48867746e-02 2.69489078e-03 ... 6.95436101e-02
  5.07026850e-02 3.03175213e-01]
 [1.37831489e-01 1.21799012e-02 1.53244856e-04 ... 9.68374842e-03
  6.82824356e-03 9.80767081e-01]
 [4.95880125e-01 3.65255024e-02 2.19329893e-03 ... 5.88213706e-02
  2.75277146e-02 1.41503157e-01]
 ...
 [5.93522962e-01 6.47160408e-02 6.13583742e-03 ... 6.85792597e-02
  2.87932860e-02 2.04425650e-01]
 [5.14593852e-01 5.49136549e-02 3.76642896e-03 ... 5.83642923e-02
  4.32955041e-02 5.27593162e-01]
 [5.21600423e-01 4.67712973e-02 1.05661668e-03 ... 3.15884817e-02
  2.04720058e-02 7.68448492e-01]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier( max_depth=6, random_state=0)

In [ ]:
y_train.reshape((y_train.size))
# y_train.tolist()
rf=clf.fit(X_train, y_train.ravel())

In [ ]:
clf.score(X_test, y_test.ravel(), sample_weight=None)

0.9252336448598131

In [ ]:
rf.feature_importances_

array([0.01192281, 0.08836499, 0.03790588, 0.02489823, 0.05203683,
       0.02004146, 0.00915199, 0.07294706, 0.02706995, 0.06331286,
       0.01649508, 0.01254572, 0.04642576, 0.08191524, 0.02739578,
       0.07776031, 0.03631933, 0.09697063, 0.06586105, 0.07051415,
       0.03198676, 0.02815812])

In [ ]:
selection = df.loc[:,'Words per sentence':]
selection.head()

,Words per sentence,Commas per sentence,Excalimation per sentence,Questions per sentence,Salutation per sentence,He per sentence,She per sentence,ACD_bigram_score,CD_bigram_score,HGW_bigram_score,JA_bigram_score,JS_bigram_score,ACD_trigram_score,CD_trigram_score,HGW_trigram_score,JA_trigram_score,JS_trigram_score,ACD_bigram_count,CD_bigram_count,HGW_bigram_count,JA_bigram_count,JS_bigram_count,Author_name_label
0,14.5785,1.1255,0.0600,0.1175,0.1205,0.1515,1.8845,12556,16106,13578,1660,1326,0,0,0,0,0,40,20,37,7,13,0
1,12.6110,0.9550,0.1050,0.0820,0.0425,0.0705,1.5245,12597,28345,13128,1544,1395,0,0,0,0,0,35,31,35,7,10,0
2,10.4260,0.8675,0.1015,0.1230,0.1050,0.0795,1.1125,12062,76079,20382,2534,1661,0,0,0,0,0,32,63,55,11,14,0
3,15.8695,1.0500,0.0750,0.0685,0.0405,0.1200,2.2515,40894,22048,11816,1323,1450,887,0,0,0,0,103,22,34,6,13,0
4,28.9245,1.8555,0.0035,0.0005,0.0765,0.0905,4.8365,191159,1829,3882,460,214,4073,0,0,0,0,453,3,10,2,2,0


In [ ]:
from sklearn.preprocessing import LabelEncoder
df.loc[:,'Author_name_label']=pd.factorize(df['Author Name'])[0].reshape(-1,1)
selection = df.loc[:,'Author_name_label']
y=selection.to_numpy()
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 